### [2] 전기차 충전소 기업 분포 <hr>

모듈 로딩) <hr>

In [1]:
# 모듈 로딩
import pandas as pd
import matplotlib.pyplot as plt
import re, os

한글 폰트 설정) <hr>

In [2]:
# ============================================================================
# 한글 폰트 설정 - 간단한 방법
# ============================================================================
plt.rcParams['font.family'] = 'Malgun Gothic'
plt.rcParams['axes.unicode_minus'] = False

print(f"현재 폰트: {plt.rcParams['font.family']}\n")

현재 폰트: ['Malgun Gothic']



데이터 로드) <hr>

In [3]:
# ============================================================================
# 필요한 데이터 로드
# ============================================================================

# 데이터 로드
charger_location = '../data/raw/charger_companies.csv'

df_charger = pd.read_csv(charger_location, encoding='cp949', low_memory=False)

print("=" * 80)
print("전기차 충전소 데이터 분석")
print("=" * 80)

print(f"\n총 데이터: {len(df_charger)}개 행")
print(f"컬럼: {list(df_charger.columns)}")
display(df_charger.head(1))

전기차 충전소 데이터 분석

총 데이터: 44개 행
컬럼: ['순위', '브랜드명', '회사명', '본사_주소', '대표자', '대표전화', '정보_확보율', 'TAGO링크']


,순위,브랜드명,회사명,본사_주소,대표자,대표전화,정보_확보율,TAGO링크
0,1,남부산전㈜,남부산전㈜,전라남도 나주시 혁신산단로 160,박일순,061-543-7744,100%,https://tago.kr/charge/company/namboo.htm


데이터 전처리) <hr>

In [4]:
# ============================================================================
# 데이터 전처리(필요없는 열 삭제)(['브랜드명', '시도']만 남겨둠)
# ============================================================================
# 브랜드명과 지역만 선택하여 지역별로 정렬
df_charger_companies = df_charger[['브랜드명', '본사_주소']].sort_values('본사_주소').reset_index(drop=True)

# ============================================================================
# 본사 주소에서 시/도 풀네임 추출
# ============================================================================

patterns = [
    (r'서울특별시|서울시|서울', '서울특별시'),
    (r'부산광역시|부산시|부산', '부산광역시'),
    (r'대구광역시|대구시|대구', '대구광역시'),
    (r'인천광역시|인천시|인천', '인천광역시'),
    (r'광주광역시|광주시|광주', '광주광역시'),
    (r'대전광역시|대전시|대전', '대전광역시'),
    (r'울산광역시|울산시|울산', '울산광역시'),
    (r'세종특별자치시|세종시|세종', '세종특별자치시'),
    (r'경기도|경기', '경기도'),
    (r'강원도|강원특별자치도|강원', '강원특별자치도'),
    (r'충청북도|충북도|충북|충청북', '충청북도'),
    (r'충청남도|충남도|충남|충청남', '충청남도'),
    (r'전라북도|전북도|전북|전라북', '전라북도'),
    (r'전라남도|전남도|전남|전라남', '전라남도'),
    (r'경상북도|경북도|경북|경상북', '경상북도'),
    (r'경상남도|경남도|경남|경상남', '경상남도'),
    (r'제주특별자치도|제주도|제주', '제주특별자치도'),
]

def get_region(address):
    if pd.isna(address) or str(address).strip() == '' or str(address).strip() == '미기입':
        return '미기입'
    
    address = str(address).strip()
    
    for pattern, region_name in patterns:
        if re.search(pattern, address):
            return region_name
    
    return '미기입'

df_charger['시도'] = df_charger['본사_주소'].apply(get_region)

# ============================================================================
# 데이터 전처리 (브랜드명과 시도만 선택하여 지역별로 정렬)
# ============================================================================
df_charger_companies = df_charger[['브랜드명', '시도']].sort_values('시도').reset_index(drop=True)

# ============================================================================
# 저장 경로 설정
# ============================================================================
save_path = '../data/processed'
os.makedirs(save_path, exist_ok=True)

# ============================================================================
# 원본 전처리 데이터 저장
# ============================================================================
csv_file = os.path.join(save_path, 'charger_companies_preprocessed.csv')
df_charger_companies.to_csv(csv_file, index=False, encoding='utf-8-sig')
print(f"✓ 전처리된 데이터 저장: {os.path.abspath(csv_file)}")

✓ 전처리된 데이터 저장: c:\Users\khw27\바탕 화면\MINI2 Pandas\data\processed\charger_companies_preprocessed.csv


시도별 충전소 기업 분포 분석) <hr>

In [5]:
# ============================================================================
# 시도별 충전소 기업 분포 분석
# ============================================================================
print("\n" + "=" * 80)
print("시도별 충전소 기업 분포")
print("=" * 80)

total_chargers = len(df_charger)                                                            # 전체 충전소 수

charger_companies_stats = df_charger['시도'].value_counts().sort_values(ascending=False)    # 오름차순 정렬

for region, count in charger_companies_stats.items():                                      # 시도(key), 개수(value)  
    percentage = (count / total_chargers) * 100                                             # 지역별 개수 / 전체 충전소 * 100
    print(f"  - {region}: {count}개 ({percentage:.1f}%)")

# ============================================================================
# 시도별 충전소 개수를 새로운 파일에 저장
# ============================================================================
# 시도별 통계 데이터프레임 생성
region_stats_df = pd.DataFrame({'시도': charger_companies_stats.index,
                                '충전소_개수': charger_companies_stats.values,
                                '비율_%': (charger_companies_stats.values / total_chargers * 100).round(1)})

# 새로운 CSV 파일로 저장
save_path = '../reports/tables'
stats_csv_file = os.path.join(save_path, 'charger_companies_stats.csv')
charger_companies_stats.to_csv(stats_csv_file, index=False, encoding='utf-8-sig')
print(f"\n✓ 시도별 충전소 기업 개수 저장: {os.path.abspath(stats_csv_file)}")


시도별 충전소 기업 분포
  - 서울특별시: 21개 (47.7%)
  - 경기도: 8개 (18.2%)
  - 제주특별자치도: 5개 (11.4%)
  - 인천광역시: 3개 (6.8%)
  - 전라남도: 2개 (4.5%)
  - 대구광역시: 2개 (4.5%)
  - 경상남도: 1개 (2.3%)
  - 광주광역시: 1개 (2.3%)
  - 충청북도: 1개 (2.3%)

✓ 시도별 충전소 기업 개수 저장: c:\Users\khw27\바탕 화면\MINI2 Pandas\reports\tables\charger_companies_stats.csv


시각화 <hr>

In [6]:
# ============================================================================
# 시각화 (matplotlib)
# ============================================================================

print("\n" + "=" * 80)
print("시각화 생성")
print("=" * 80)


# 1. 시도별 충전소 기업 분포 (막대그래프)
fig1, ax1 = plt.subplots(figsize=(8, 6))    # fig(그림판 전체), ax(그래프 영역)

charger_companies_stats = charger_companies_stats.sort_values(ascending=True)  # 내림차순 정렬
 
# barh(x축, y축, 막대 색상): 가로 막대 그래프 그리는 함수
bars = ax1.barh(charger_companies_stats.index, charger_companies_stats.values, color='skyblue')  

ax1.set_title('시도별 충전소 기업 분포', fontsize=12, fontweight='bold')
ax1.set_xlabel('개수')

for bar in bars:                # bars: 각 막대 객체들의 리스트

    width = bar.get_width()     # 막대의 너비(width): 충전소의 개수

    # 막대에 텍스트 추가
    ax1.text(width, bar.get_y() + bar.get_height()/2,   # 텍스트의 위치: width(막대의 오른쪽 끝), 
                                                          #              bar.get_y(): 막대의 시작 y좌표(가로 막대 기준 막대의 아래쪽 위치)
                                                          #              bar.get_height(): 막대의 높이(굵기)
             f'{int(width):,}',     # 표시형식(천 단위 구분 콤마 포함)
             ha='left', va='center', fontsize=9, fontweight='bold') # 텍스트 정렬: 왼쪽, 수직 중앙, 글자크기, 글씨체

fig1.tight_layout() # 그래프의 요소들 겹치지 않게 자동 조정
fig1.savefig('../reports/figures/2_01_charger_companies_stats.png', dpi=300)
plt.close(fig1)     # 그래프 닫음(메모리 절약)
print("✓ 그래프 저장: reports/figures/2_01_charger_companies_stats.png")


# 2. 시도별 충전소 기업 비율 (막대그래프)
fig2, ax2 = plt.subplots(figsize=(8, 6))

total_count = charger_companies_stats.sum()
region_stats_percent = (charger_companies_stats / total_count * 100).sort_values(ascending=True)

bars2 = ax2.barh(region_stats_percent.index, region_stats_percent.values, color='coral')

ax2.set_title('시도별 충전소 기업 비율', fontsize=12, fontweight='bold')
ax2.set_xlabel('비율 (%)')

for bar in bars2:

    width = bar.get_width()

    ax2.text(width, bar.get_y() + bar.get_height()/2, 
             f'{width:.1f}%',
             ha='left', va='center', fontsize=9, fontweight='bold')

fig2.tight_layout()
fig2.savefig('../reports/figures/2_02_charger_companies_ratio.png', dpi=300)
plt.close(fig2)
print("✓ 그래프 저장: reports/figures/2_02_charger_companies_ratio.png")


시각화 생성
✓ 그래프 저장: reports/figures/2_01_charger_companies_stats.png
✓ 그래프 저장: reports/figures/2_02_charger_companies_ratio.png
